# Leafletfinder to Edge List

Use Leaflet Finder Code to create an graph based on Oliver's MD trajectory.

Info from Mail:

We uploaded a test system for you via our file share service and you should have received a barrage of emails from the service (apologies!).

It is a system with 1.7 M particles and almost 150,000 lipids (i.e. 150,000 nodes in the network). There's also a mini python script that shows how to run it. Please note that we just fixed a bug in the MDAnalysis leaflet finder code that crept in one release ago. It is fixed in the 0.12.1 release that will be out tomorrow (or get the develop branch from github).

We tried running the basic version of leaflet finder on it but got a MemoryError; apparently, it tries to allocate 2 TiB of RAM... The "sparse" option works

        L = LeafletFinder(u, "name P*", sparse=True, pbc=True)

but took over 4 min for a single frame (and a bit over 1 min with pbc=False)  so this is too slow (and pbc=True is typically necessary).

The trajectory itself only contains 15 frames (1/1000th of the original one) but our file sharing service does not lik 80 GiB files...

The trajectory is not the nicest example yet in terms of the kind of fusion between vesicles that we want to observe but it should give you something to play with.

**Data:**

In [ ]:
import numpy as np
np.__config__.show() 

The first step in any MDAnalysis script is to load a topology file (which contains a list of particles and possibly additional static data such as bonds or partial charges) and a trajectory file. The trajectory contains the coordinates, which change for each time step. In MDAnalysis, the Universe object ties the topology and the trajectory together and part of
the process of instantiating Universe (topology, trajectory ) is to parse these files.

Source: <http://dx.doi.org/10.6084/m9.figshare.1588804>

File Types:
* `.xtc` compressed trajectory file from Gromacs
* `.tpr` topology files

In [13]:
!ls -lh /data/leafletfinder/large/vesicle_1_5M_373*

-rw-r--r-- 1 iparask iparask 71M Nov  4 20:22 /data/leafletfinder/large/vesicle_1_5M_373_first.gro
-rw-r--r-- 1 iparask iparask 71M Nov  4 20:20 /data/leafletfinder/large/vesicle_1_5M_373_last.gro
-rw-r--r-- 1 iparask iparask 90M Nov  4 20:21 /data/leafletfinder/large/vesicle_1_5M_373_stride1000.xtc
-rw-r--r-- 1 iparask iparask 41M Nov  4 20:22 /data/leafletfinder/large/vesicle_1_5M_373.tpr


In [1]:
import MDAnalysis, time
topology = "/data/leafletfinder/large/vesicle_1_5M_373.tpr"
trajectory = "/data/leafletfinder/large/vesicle_1_5M_373_stride1000.xtc"

start = time.time()
u = MDAnalysis.Universe(topology, trajectory)
print "Loading Time: %.2f"%(time.time()-start)

Loading Time: 14.42


In [2]:
%time
start = time.time()
selection = u.select_atoms("name P*")
print "Selection Time: %.2f"%(time.time()-start)

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 9.06 µs
Selection Time: 3.03


In [22]:
u

<Universe with 1748952 atoms and 1603206 bonds>

In [29]:
count = 0
for ts in u.trajectory:
    print("Frame: %5d, Time: %8.3f ps" % (ts.frame, u.trajectory.time))
    print("Rgyr: %g A" % (u.atoms.radius_of_gyration(), ))
    count = count + 1 
print "Number of frames: %d"%count 

Frame:     0, Time:    0.000 ps
Rgyr: 652.801 A
Frame:     1, Time: 50000.000 ps
Rgyr: 650.131 A
Frame:     2, Time: 100000.000 ps
Rgyr: 637.096 A
Frame:     3, Time: 150000.000 ps
Rgyr: 627.282 A
Frame:     4, Time: 200000.000 ps
Rgyr: 618.614 A
Frame:     5, Time: 250000.000 ps
Rgyr: 609.713 A
Frame:     6, Time: 300000.000 ps
Rgyr: 599.82 A
Frame:     7, Time: 350000.000 ps
Rgyr: 588.657 A
Frame:     8, Time: 400000.000 ps
Rgyr: 578.532 A
Frame:     9, Time: 450000.000 ps
Rgyr: 564.654 A
Frame:    10, Time: 500000.000 ps
Rgyr: 550.324 A
Frame:    11, Time: 550000.000 ps
Rgyr: 533.978 A
Frame:    12, Time: 600000.000 ps
Rgyr: 516.298 A
Frame:    13, Time: 650000.000 ps
Rgyr: 499.393 A
Frame:    14, Time: 700000.000 ps
Rgyr: 483.763 A
Number of frames: 15


In [23]:
selection

<AtomGroup with 145746 atoms>

In [ ]:
import MDAnalysis.analysis.leaflet
start=time.time()
L = MDAnalysis.analysis.leaflet.LeafletFinder(u, "name P*", pbc=False, sparse=True)
print "Create Graph Time: %.2f"%(time.time()-start)

In [ ]:
import networkx as NX
start = time.time()
graph = L.graph
cc = NX.connected_components(graph)
count = 0
for i in cc:
    count = count + 1
print str(count)
print "CC Time: %.2f"%(time.time()-start)

In [ ]:
NX.write_edgelist(graph,
                  "graph_edges_%d_%d.csv"%(NX.number_of_nodes(graph), NX.number_of_edges(graph)),
                  delimiter=",")